In [ ]:
# --- 1. Verify and install dependencies ---
print("Verifying and installing missing packages if necessary...")

packages_to_check = [
    'numpy', 'matplotlib', 'librosa', 'tqdm', 'sklearn', 'stempeg', 'torch', 'torchvision', 'torchaudio', 'musdb', 'nbformat', 'nbconvert'
]

for package in packages_to_check:
    try:
        __import__(package)
        print(f"  ✅ {package} is installed.")
    except ImportError:
        print(f"  ❌ {package} is NOT installed. Attempting to install...")
        try:
            import sys
            import subprocess
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            __import__(package)
            print(f"  ✅ {package} is now installed.")
        except Exception as e:
            print(f"  ❌ Failed to install {package}: {e}")

# Special check for PyTorch CUDA
print("\n--- PyTorch CUDA status ---")
try:
    import torch
    if torch.cuda.is_available():
        print(f"  ✅ PyTorch with CUDA (version {torch.version.cuda}) is available.")
        print(f"     CUDA Device Name: {torch.cuda.get_device_name(0)}")
    else:
        print("  ⚠️ PyTorch is installed, but CUDA is NOT available.")
except ImportError:
    print("  ❌ PyTorch is not installed.")

print("Verification complete.")

# --- 2. Set experiment configs for model_A.ipynb ---
MODEL_CONFIG = {
    'in_channels': 1,
    'out_channels': 1,
    'base_filters': 64,
    'num_layers': 4,
    'batchnorm': True,
    'dropout': 0.1,
}
TRAIN_CONFIG = {
    'num_epochs': 50,
    'learning_rate': 1e-4,
    'patience': 10000,
    'batch_size': 2,
}
OVERFIT_CONFIG = {
    'base_filters': 128,
    'num_layers': 4,
    'batchnorm': True,
    'dropout': 0.0,
    'learning_rate': 3e-4,
    'num_epochs': 100,
    'batch_size': 2,
    'patience': 10000,
}


# Main Experiment Controller

This notebook lets you control model architecture and training hyperparameters for model_A.ipynb from a single place.

**Workflow:**
1. Set your experiment configs below (MODEL_CONFIG, TRAIN_CONFIG, OVERFIT_CONFIG).
2. Run all cells to verify dependencies and execute model_A.ipynb with your chosen parameters.
3. To try different experiments, just change the config values and re-run.

**Example:**
- Change `base_filters` or `batch_size` in the config dicts to test different model sizes or memory usage.
- All results, checkpoints, and logs will be produced as usual by model_A.ipynb.


In [ ]:
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

# --- 3. Find and run model_A.ipynb with injected configs ---

def find_notebook(filename, search_path="."):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

notebook_path = find_notebook("model_A.ipynb", ".")
if notebook_path is None:
    raise FileNotFoundError("model_A.ipynb not found anywhere in the workspace.")

with open(notebook_path) as f:
    nb = nbformat.read(f, as_version=4)

# Remove autoreload magic commands from the notebook before execution
for cell in nb.cells:
    if cell.cell_type == 'code':
        lines = cell.source.splitlines()
        new_lines = [line for line in lines if not line.strip().startswith(('%load_ext autoreload', '%autoreload'))]
        cell.source = '\n'.join(new_lines)

# Inject config variables into the first code cell
inject_code = [
    "MODEL_CONFIG = globals().get('MODEL_CONFIG', MODEL_CONFIG)",
    "TRAIN_CONFIG = globals().get('TRAIN_CONFIG', TRAIN_CONFIG)",
    "OVERFIT_CONFIG = globals().get('OVERFIT_CONFIG', OVERFIT_CONFIG)"
]
for cell in nb.cells:
    if cell.cell_type == 'code':
        cell.source = '\n'.join(inject_code) + '\n' + cell.source
        break

# Execute the notebook
ep = ExecutePreprocessor(timeout=1200, kernel_name='python3')
try:
    ep.preprocess(nb, {'metadata': {'path': os.path.dirname(notebook_path)}})
    print(f'model_A.ipynb executed successfully from: {notebook_path}')
except Exception as e:
    print(f'Error executing model_A.ipynb: {e}')